In [113]:
#import necessary libraries
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
from pandas.tseries.offsets import BDay
import random
import math
import numpy as np
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from scipy.stats import norm
import plotly.graph_objects as go
import warnings

warnings.filterwarnings('ignore')

First lets establish a prophet model and use it to predict the market

In [114]:
# We are analyzing the S&P 500 Index thru the prophet API,
# thus we need to garner the relevant data using yfinance

today = datetime.today().strftime('%Y-%m-%d')
start = '2016-01-01'

sp_df = yf.download('^GSPC', start, today)
sp_df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-01-04,2038.199951,2038.199951,1989.680054,2012.660034,2012.660034,4304880000
2016-01-05,2013.780029,2021.939941,2004.170044,2016.709961,2016.709961,3706620000
2016-01-06,2011.709961,2011.709961,1979.050049,1990.260010,1990.260010,4336660000
2016-01-07,1985.319946,1985.319946,1938.829956,1943.089966,1943.089966,5076590000
2016-01-08,1945.969971,1960.400024,1918.459961,1922.030029,1922.030029,4664940000
...,...,...,...,...,...,...
2022-08-15,4269.370117,4301.790039,4256.899902,4297.140137,4297.140137,3087740000
2022-08-16,4290.459961,4325.279785,4277.770020,4305.200195,4305.200195,3792010000
2022-08-17,4280.399902,4302.180176,4253.080078,4274.040039,4274.040039,3293430000


In [115]:
# data checks
sp_df.info()
sp_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1670 entries, 2016-01-04 to 2022-08-19
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1670 non-null   float64
 1   High       1670 non-null   float64
 2   Low        1670 non-null   float64
 3   Close      1670 non-null   float64
 4   Adj Close  1670 non-null   float64
 5   Volume     1670 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 91.3 KB


Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [116]:
# prophet is a time series model so we must index by date
sp_df.reset_index(inplace = True)
sp_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [117]:
# prophet models require soley a date and y component
# here that y will be Adj Close

df = sp_df[['Date', 'Adj Close']]

# rename for prophet usage
df.rename(columns = {'Date' : 'ds', 'Adj Close' : 'y'}, inplace = True)
df

,ds,y
0,2016-01-04,2012.660034
1,2016-01-05,2016.709961
2,2016-01-06,1990.260010
3,2016-01-07,1943.089966
4,2016-01-08,1922.030029
...,...,...
1665,2022-08-15,4297.140137
1666,2022-08-16,4305.200195
1667,2022-08-17,4274.040039
1668,2022-08-18,4283.740234


In [118]:
# quickly vizualizing data before prophet use
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x = df['ds'], y = df['y']))

fig1.update_layout(
    title_text = "S&P 500 Pricing with Prophet"
)

# adding sliders and buttons for interaction
fig1.update_layout(
    xaxis = dict(
        rangeselector = dict(
            buttons = list([
                dict(
                    count = 1,
                    label = '1m',
                    step = 'month',
                    stepmode = 'backward'
                ),
                dict(
                    count = 6,
                    label = '6m',
                    step = 'month',
                    stepmode = 'backward'
                ),
                dict(
                    count = 1,
                    label = 'YTD',
                    step = 'year',
                    stepmode = 'todate'
                ),
                dict(
                    count = 1,
                    label = '1y',
                    step = 'year',
                    stepmode = 'backward'
                ),
                dict(
                    count = 5,
                    label = '5y',
                    step = 'year',
                    stepmode = 'backward'
                ),
                dict(step = 'all')
            ])
        ),
        rangeslider = dict(
            visible =True
        ),
        type = 'date'
    )
)

fig1.show()

In [119]:
# build prophet model for price forcasting
m = Prophet(seasonality_mode='multiplicative')
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


In [120]:
# here we create a year's worth of time series for the prophet model
future = m.make_future_dataframe(periods = 365)
business_days = BDay().onOffset
filter = pd.to_datetime(future['ds']).map(business_days)
future = future[filter]
future.tail(10)

,ds
2022,2023-08-07
2023,2023-08-08
2024,2023-08-09
2025,2023-08-10
2026,2023-08-11
2029,2023-08-14
2030,2023-08-15
2031,2023-08-16
2032,2023-08-17
2033,2023-08-18


In [121]:
# finally, we get to our predictions
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(10)

,ds,yhat,yhat_lower,yhat_upper
1920,2023-08-07,3996.514034,1388.765076,6905.336003
1921,2023-08-08,4002.500831,1308.919087,6850.570307
1922,2023-08-09,4009.206353,1311.708239,6940.372163
1923,2023-08-10,4011.289926,1330.307553,6993.314566
1924,2023-08-11,4017.740633,1331.885121,6961.557577
1925,2023-08-14,4030.340491,1277.479050,7036.378007
1926,2023-08-15,4037.021174,1248.611289,7115.638207
1927,2023-08-16,4044.261944,1307.950989,7099.599734
1928,2023-08-17,4046.704748,1243.800411,7087.216902
1929,2023-08-18,4053.313018,1226.455289,7174.924680


In [122]:
# lets specifically get the next days worth!
next_day = ((datetime.today() + timedelta(1)).strftime('%Y-%m-%d'))
forecast[forecast['ds'] == next_day]

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
1670,2022-08-22,761.736626,4147.788873,4354.403116,761.736626,761.736626,4.581646,4.581646,4.581646,4.58405,4.58405,4.58405,-0.002403,-0.002403,-0.002403,0.0,0.0,0.0,4251.74454


In [123]:
plot_plotly(m, forecast)

Now, lets approach this problem with a more traditional/basic method in finance: Monte Carlo Simulation

In [124]:
data = pd.DataFrame()
data = sp_df['Adj Close']
data

0       2012.660034
1       2016.709961
2       1990.260010
3       1943.089966
4       1922.030029
           ...     
1665    4297.140137
1666    4305.200195
1667    4274.040039
1668    4283.740234
1669    4228.479980
Name: Adj Close, Length: 1670, dtype: float64

In [125]:
log_returns = np.log(1 + data.pct_change())
log_returns

0            NaN
1       0.002010
2      -0.013202
3      -0.023986
4      -0.010898
          ...   
1665    0.003962
1666    0.001874
1667   -0.007264
1668    0.002267
1669   -0.012984
Name: Adj Close, Length: 1670, dtype: float64

In [126]:
#returns stats
u = log_returns.mean()
var = log_returns.var()
drift = u - (0.5 * var)
stdev = log_returns.std()

# trading days per year and num of sims
t_intervals = 252
iterations = 10

daily_returns = np.exp(drift + stdev * norm.ppf(np.random.rand(t_intervals, iterations)))

s0 = data.iloc[-1]
s0

4228.47998046875

In [127]:
price_list = np.zeros_like(daily_returns)
price_list[0] = s0
price_list

array([[4228.47998047, 4228.47998047, 4228.47998047, ..., 4228.47998047,
        4228.47998047, 4228.47998047],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       ...,
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ]])

In [128]:
for t in range(1, t_intervals):
    price_list[t] = price_list[t-1] * daily_returns[t]

price_list

columns = []
for i in range(0,10):
     columns.append("sim" + str(i))

df1 = pd.DataFrame(price_list, columns = columns)
df1['date'] = pd.date_range(start= today, periods=len(df1), freq=BDay())
df1

,sim0,sim1,sim2,sim3,sim4,sim5,sim6,sim7,sim8,sim9,date
0,4228.479980,4228.479980,4228.479980,4228.479980,4228.479980,4228.479980,4228.479980,4228.479980,4228.479980,4228.479980,2022-08-22
1,4238.852174,4183.764976,4278.747087,4295.824431,4200.089527,4178.489780,4319.973724,4256.032664,4311.515619,4278.447327,2022-08-23
2,4267.406485,4142.184873,4261.890016,4454.443844,4216.088321,4180.395003,4405.160436,4241.444438,4306.540496,4238.094440,2022-08-24
3,4249.679944,4062.279013,4226.157011,4486.092770,4236.421004,4209.368171,4428.277582,4298.806130,4313.013689,4161.596172,2022-08-25
4,4238.648691,3974.023232,4247.665665,4485.857221,4325.096375,4179.228065,4441.286537,4286.762439,4357.429741,4166.108122,2022-08-26
...,...,...,...,...,...,...,...,...,...,...,...
247,4703.722332,5746.595722,3853.914072,3842.292056,4921.967984,4807.322907,4037.755981,3756.187903,6846.083168,4280.212514,2023-08-02
248,4779.603319,5788.897660,3905.572032,3879.947068,4919.753557,4799.756413,4032.457185,3771.576756,6799.422214,4247.859630,2023-08-03
249,4744.984318,5713.719487,3843.594270,3908.433286,4870.182703,4823.419469,4132.323058,3854.219271,6925.962486,4335.335277,2023-08-04
250,4765.155799,5713.388951,3819.302113,3893.983004,4961.602974,4799.782620,4104.062941,3793.495913,6866.790156,4342.783113,2023-08-07


In [129]:
# Let's visualize the Monte Carlo Simulations
fig2 = go.Figure()
for name in columns:
    fig2.add_trace(go.Scatter(x = df1['date'], y = df1[name]))

fig2.update_layout(
    title_text = "S&P 500 Pricing with Monte Carlo"
)

# adding sliders and buttons for interaction
fig2.update_layout(
    xaxis = dict(
        rangeselector = dict(
            buttons = list([
                dict(
                    count = 1,
                    label = '1m',
                    step = 'month',
                    stepmode = 'backward'
                ),
                dict(
                    count = 6,
                    label = '6m',
                    step = 'month',
                    stepmode = 'backward'
                ),
                dict(
                    count = 1,
                    label = 'YTD',
                    step = 'year',
                    stepmode = 'todate'
                ),
                dict(
                    count = 1,
                    label = '1y',
                    step = 'year',
                    stepmode = 'backward'
                ),
                dict(
                    count = 5,
                    label = '5y',
                    step = 'year',
                    stepmode = 'backward'
                ),
                dict(step = 'all')
            ])
        ),
        rangeslider = dict(
            visible =True
        ),
        type = 'date'
    )
)

fig2.show()

For another comparison, lets approach Monte Carlo in a more nuanced way: Euler Discretization

In [130]:
# annual stdev of returns
stdev_a = log_returns.std() * 252 ** 0.5

# risk-free rate of return (bonds)
r = 0.026

# time horizon (years)
T = 1

# intervals within horizon (trading days)
t_intervals = 252
delta_t = t / t_intervals

In [131]:
Z = np.random.standard_normal((t_intervals + 1, iterations))
S = np.zeros_like(Z)
S0 = data.iloc[-1]
S[0] = S0
S

array([[4228.47998047, 4228.47998047, 4228.47998047, ..., 4228.47998047,
        4228.47998047, 4228.47998047],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       ...,
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ]])

We need to use the following recursive formula for simulation:
$$
S_t = S_{t-1} \cdot exp((r - 0.5 \cdot stdev^2) \cdot delta_t + stdev \cdot delta_t^{0.5} \cdot Z_t)
$$

In [132]:
for t in range(1, t_intervals + 1):
    S[t] = S[t-1] * np.exp((r - 0.5 * stdev ** 2) * delta_t + stdev * delta_t ** 0.5 * Z[t])
S

array([[   4228.47998047,    4228.47998047,    4228.47998047, ...,
           4228.47998047,    4228.47998047,    4228.47998047],
       [   4386.6196515 ,    4448.67529495,    4326.65921466, ...,
           4322.10956697,    4289.75978654,    4255.14378458],
       [   4497.23740747,    4552.87048118,    4495.76928315, ...,
           4414.39656511,    4473.47869291,    4413.84710898],
       ...,
       [3007319.63492598, 3839050.96578938, 2545531.75228886, ...,
        3029457.13499686, 3026279.7296755 , 1834338.4810607 ],
       [3104122.95247288, 4033983.65142658, 2571921.30456657, ...,
        3169602.51189913, 3101218.73529946, 1887507.75447942],
       [3201744.22147523, 4099598.55411754, 2619081.51883221, ...,
        3201062.17815766, 3174412.24832667, 1924575.10151527]])

In [133]:
df2 = pd.DataFrame(S, columns = columns)
df2['date'] = pd.date_range(start= today, periods=len(df2), freq=BDay())
df2

,sim0,sim1,sim2,sim3,sim4,sim5,sim6,sim7,sim8,sim9,date
0,4.228480e+03,4.228480e+03,4.228480e+03,4.228480e+03,4.228480e+03,4.228480e+03,4.228480e+03,4.228480e+03,4.228480e+03,4.228480e+03,2022-08-22
1,4.386620e+03,4.448675e+03,4.326659e+03,4.247506e+03,4.227678e+03,4.400741e+03,4.475473e+03,4.322110e+03,4.289760e+03,4.255144e+03,2022-08-23
2,4.497237e+03,4.552870e+03,4.495769e+03,4.405901e+03,4.386541e+03,4.488848e+03,4.643523e+03,4.414397e+03,4.473479e+03,4.413847e+03,2022-08-24
3,4.606618e+03,4.761084e+03,4.629196e+03,4.605642e+03,4.415791e+03,4.641659e+03,4.847887e+03,4.644083e+03,4.608072e+03,4.635215e+03,2022-08-25
4,4.733256e+03,4.918861e+03,4.949029e+03,4.610455e+03,4.524923e+03,4.727638e+03,4.951308e+03,4.756585e+03,4.630916e+03,4.695202e+03,2022-08-26
...,...,...,...,...,...,...,...,...,...,...,...
248,2.879777e+06,3.582396e+06,2.437061e+06,2.703589e+06,2.359920e+06,2.752315e+06,3.183005e+06,2.861130e+06,2.861992e+06,1.741968e+06,2023-08-03
249,2.938075e+06,3.686473e+06,2.455715e+06,2.735060e+06,2.379359e+06,2.869903e+06,3.275307e+06,2.955368e+06,2.934203e+06,1.794958e+06,2023-08-04
250,3.007320e+06,3.839051e+06,2.545532e+06,2.844183e+06,2.427817e+06,2.945194e+06,3.362684e+06,3.029457e+06,3.026280e+06,1.834338e+06,2023-08-07
251,3.104123e+06,4.033984e+06,2.571921e+06,3.008481e+06,2.488009e+06,3.059447e+06,3.470046e+06,3.169603e+06,3.101219e+06,1.887508e+06,2023-08-08


In [134]:
# Let's visualize the Euler Discretizations
fig3 = go.Figure()
for name in columns:
    fig3.add_trace(go.Scatter(x = df1['date'], y = df1[name]))

fig3.update_layout(
    title_text = "S&P 500 Pricing with Euler Discretization"
)

# adding sliders and buttons for interaction
fig3.update_layout(
    xaxis = dict(
        rangeselector = dict(
            buttons = list([
                dict(
                    count = 1,
                    label = '1m',
                    step = 'month',
                    stepmode = 'backward'
                ),
                dict(
                    count = 6,
                    label = '6m',
                    step = 'month',
                    stepmode = 'backward'
                ),
                dict(
                    count = 1,
                    label = 'YTD',
                    step = 'year',
                    stepmode = 'todate'
                ),
                dict(
                    count = 1,
                    label = '1y',
                    step = 'year',
                    stepmode = 'backward'
                ),
                dict(
                    count = 5,
                    label = '5y',
                    step = 'year',
                    stepmode = 'backward'
                ),
                dict(step = 'all')
            ])
        ),
        rangeslider = dict(
            visible =True
        ),
        type = 'date'
    )
)

fig3.show()